# Particle MCMC model for COVID-19 in Mexico

This notebook contains all of the Python code needed to use Particle Marginal Markov Chain Monte Carlo to explore the parameter space of our diffusion driven model and estimate the trajectories of the hidden states

In [11]:
# Import all necesary libraries
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

In [651]:
data = pd.read_feather('/Users/ro/Desktop/Undergrad_AM_Thesis/Data/covid_df.feather')
weekly_covid_df = data.groupby([pd.Grouper(key='date', freq='W-SUN')]).agg({
    'confirmed_cases': 'sum',
    'confirmed_deaths': 'sum'
}).reset_index()
weekly_covid_df.head()

,date,confirmed_cases,confirmed_deaths
0,2020-02-23,4.0,0.0
1,2020-03-01,23.0,0.0
2,2020-03-08,131.0,0.0
3,2020-03-15,624.0,0.0
4,2020-03-22,1152.0,6.0


In [652]:
pop = pd.read_feather('/Users/ro/Desktop/Undergrad_AM_Thesis/Data/INEGI_2020_State_Population.feather')
N = pop['population'].sum()
print('Total Population: ', N)
print('0.1% of Population: ', np.round(0.001*N).astype(int))


Total Population:  126014024
0.1% of Population:  126014


In [997]:
# Deifine prior for each parameter

def generate_prior_alpha():
    return stats.beta(2.3, 96.5).rvs()

def generate_prior_rho():
    return stats.beta(7.2, 0.8).rvs()

def generate_prior_nu():
    return stats.beta(6.9, 14.1).rvs()

def generate_prior_gamma():
    return stats.beta(2.36, 9.44).rvs()

def generate_prior_sigma_b():
    return stats.invgamma(a=3.8, scale=0.6).rvs()

def generate_prior_sigma_m():
    return stats.invgamma(a=3.8, scale=0.6).rvs()


In [1140]:
# Initial Value Conditions Prior
def set_IVC(N, rho, nu, num_particles):
    R = np.zeros(num_particles)
    D = np.zeros(num_particles)
    E = np.round(stats.uniform(0, np.round(0.001 * N).astype(int)).rvs(num_particles))
    U = np.round(stats.uniform(0, np.round((1 - rho) * nu * E).astype(int)).rvs(num_particles))
    O = np.round(stats.uniform(0, np.round(rho * nu * E).astype(int)).rvs(num_particles))
    E -=  U + O 
    S = N - E - U - O
    
    beta = stats.uniform(0,1).rvs(num_particles)
    mu = stats.uniform(0,1).rvs(num_particles)
    
    return np.vstack([S, E, U, O, R, D, np.log(beta), np.log(mu)])

def set_param_priors():
    alpha = stats.beta(2.3, 96.5).rvs()
    rho = stats.beta(7.2, 0.8).rvs()
    nu = stats.beta(6.9, 14.1).rvs()
    gamma = stats.beta(2.36, 9.44).rvs()
    sigma_b = stats.invgamma(a=3.8, scale=0.6).rvs()
    sigma_m = stats.invgamma(a=3.8, scale=0.6).rvs()
    psi_o = stats.gamma(50,scale=100).rvs()
    psi_d = stats.gamma(50,scale=100).rvs()
    
    return [alpha, rho, nu, gamma, sigma_b, sigma_m, psi_o, psi_d]
    

In [ ]:
T = 50*7
h = 0.5
num_steps = int(T / h)
num_particles=1

# Initialize arrays to store results
time = np.linspace(0, T, num_steps)
X = np.zeros([8,num_particles,num_steps])

# Set initial conditions
alpha, rho, nu, gamma, sigma_b, sigma_m, psi_o, psi_d = set_param_priors()
X[:,:,0] = set_IVC(N, rho, nu, num_particles)


beta[0] = np.log(X[6,:,0])


# Simulate the SIR model with stochastic beta using Euler-Maruyama method
for t in range(1, num_steps):
# Update the SIR compartments using Euler's method with the updated beta_t
S[t] = S[t-1] + - beta[t-1] * (S[t-1] * I[t-1] / N) * h
I[t] = I[t-1] + (beta[t-1] * (S[t-1] * I[t-1] / N) - gamma * I[t-1]) * h
R[t] = R[t-1] + gamma * I[t-1] * h 

# Sample Brownian increment
dB = stats.norm().rvs(1)[0]

# Update X using Euler-Maruyama method
X[t] = X[t-1] + sigma * np.sqrt(h) * dB

# Update beta (infection rate) as a log-transformation of X
beta[t] = np.log(X[t])